In [8]:
%matplotlib inline
%config InlineBackend.figure_formats = ["svg"]

In [9]:
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
T = 15.0 + 273.15
P = ct.one_atm

X_fuel = {
    "CH4": 92.19,
    "C2H6": 4.27,
    "C3H8": 0.64,
    "N2": 1.40,
    "CO2": 0.95
}

X_oxid = {
    "O2": 1.0
}

sum(X_fuel.values())

99.45

In [11]:
gas = ct.Solution("gri30.yaml")

gas.TP = T, P
gas.set_equivalence_ratio(1, X_fuel, X_oxid, basis="mole")
gas.mole_fraction_dict()

{'C2H6': 0.014140243397632252,
 'C3H8': 0.0021193807434390266,
 'CH4': 0.3052901730275685,
 'CO2': 0.003145955791042305,
 'N2': 0.004636145376272869,
 'O2': 0.670668101664045}

In [12]:
print(gas.report())


  gri30:

       temperature   288.15 K
          pressure   1.0132e+05 Pa
           density   1.148 kg/m^3
  mean mol. weight   27.145 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -9.4806e+05       -2.5735e+07  J
   internal energy       -1.0363e+06       -2.8131e+07  J
           entropy            7551.5        2.0498e+05  J/K
    Gibbs function        -3.124e+06       -8.4801e+07  J
 heat capacity c_p            1163.5             31582  J/K
 heat capacity c_v            857.17             23268  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                O2           0.79058           0.67067           -25.075
               CH4           0.18043           0.30529           -54.742
               CO2         0.0051004          0.003146           -195.7

In [13]:
y_fuel = gas["CH4"].Y[0]

u0 = gas.int_energy_mass
h0 = gas.enthalpy_mass

gas.equilibrate("HP")

u1 = gas.int_energy_mole

gas.TP = T, P

h1 = gas.enthalpy_mass

du = -(u1 - u0) / 1e6
lhv = -(h1 - h0) / (y_fuel * 1e6)

print(du, lhv)

44.884976945063734 28.77610183341895


In [14]:
# Set reactants state
gas.TPX = 298, 101325, "CH4:1, O2:2"
h1 = gas.enthalpy_mass
Y_CH4 = gas["CH4"].Y[0]  # returns an array, of which we only want the first element

# set state to complete combustion products without changing T or P
gas.TPX = None, None, "CO2:1, H2O:2"
h2 = gas.enthalpy_mass

LHV = -(h2 - h1) / Y_CH4 / 1e6
print(f"LHV = {LHV:.3f} MJ/kg")

LHV = 50.025 MJ/kg
